In [1]:
import json
import re
from random import seed, randrange
from math import log
from sklearn.utils import shuffle

import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cyana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1 Import Dataset

In [2]:
data_df = pd.read_csv('data/bgg-13m-reviews.csv',index_col=0)
data_df.head()

D:\_software\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user,rating,comment,ID,name
0,sidehacker,10.0,NaN,13,Catan
1,Varthlokkur,10.0,NaN,13,Catan
2,dougthonus,10.0,"Currently, this sits on my list as my favorite...",13,Catan
3,cypar7,10.0,"I know it says how many plays, but many, many ...",13,Catan
4,ssmooth,10.0,NaN,13,Catan


# 2 Data Preprocessing

In [3]:
data_df = data_df.dropna()
data_df.head()

,user,rating,comment,ID,name
2,dougthonus,10.0,"Currently, this sits on my list as my favorite...",13,Catan
3,cypar7,10.0,"I know it says how many plays, but many, many ...",13,Catan
7,hreimer,10.0,i will never tire of this game.. Awesome,13,Catan
11,daredevil,10.0,This is probably the best game I ever played. ...,13,Catan
16,hurkle,10.0,Fantastic game. Got me hooked on games all ove...,13,Catan


In [4]:
reviews_df = data_df[['rating','comment']]
reviews_df.head()

,rating,comment
2,10.0,"Currently, this sits on my list as my favorite..."
3,10.0,"I know it says how many plays, but many, many ..."
7,10.0,i will never tire of this game.. Awesome
11,10.0,This is probably the best game I ever played. ...
16,10.0,Fantastic game. Got me hooked on games all ove...


In [5]:
reviews_df = shuffle(reviews_df)
reviews_df.head()

,rating,comment
3124024,4.0,Beautiful but way too long and unexciting
9171441,6.0,played twice. second time was bad. slow and d...
7840203,9.0,Some of our favorite scenarios/missions have b...
6581853,6.0,Not really too bad for a worker-placement game.
5211869,4.0,So now we have seperate entries for WotR first...


## 2.1 Segmentation

In [6]:
def segmentation(str):
    words = re.sub('[^a-zA-Z]',' ', str).lower().split() # Remove non-alphabetic characters
    stop_words = (nltk.corpus.stopwords.words('english')) # Remove stopwords
    words =  [x for x in words if x not in stop_words]
    return words

In [7]:
x = [segmentation(review) for review in reviews_df['comment']]
y = [round(r) for r in reviews_df['rating']]

# 3 Divide Data

In [8]:
x_train, y_train, x_test, y_test = x, y, [], []
test_size = int(len(x)*0.01)

seed(1)

for _ in range(test_size):
    random_index = randrange(len(x_train))
    x_test.append(x_train.pop(random_index))
    y_test.append(y_train.pop(random_index))

print('Size of Train Set: ', len(x_train))
print('Size of Test Set: ', len(x_test))

Size of Train Set:  2611379
Size of Test Set:  26377


# 4 Text Feature Extraction

In [9]:
# Get all the words in the training set non-repeatedly and record the index of each word
words_index_dict = dict()
index = 0
for rating in x_train:
    for word in rating:
        if word not in words_index_dict:
            words_index_dict[word]=index
            index+=1

In [10]:
tf={}
idf = [0 for _ in range(len(words_index_dict))]
for review_index, review in enumerate(x_train):
    review_counts = pd.value_counts(review)
    for word_index, word in enumerate(review):
        if word in words_index_dict:
            tf[(review_index,words_index_dict[word])] = review_counts[word]/len(review)
            idf[words_index_dict[word]]+=1

idf = [log(len(x_train)/(cont+1)) for cont in idf]

# 5 Generate Conditional Probability Matrix

In [11]:
for key in tf:
    tf[key]*=idf[key[1]]

In [12]:
tfidf=dict()
for rating in range(11):
    tfidf[rating]=[0 for _ in range(len(words_index_dict))]

In [13]:
for key, value in tf.items():
    label = y_train[key[0]]
    word_index = key[1]
    tfidf[label][word_index]+=value

In [14]:
for i in range(len(tfidf)):
    row_sum = sum(tfidf[i])
    tfidf[i]=[x/row_sum for x in tfidf[i]]

# 6 Calculate Accuracy

In [15]:
label_count = [0 for _ in range(11)] + [len(x_train)]
for rating in y_train:
    label_count[rating]+=1

In [16]:
def count_value(list):
    value_count=dict()
    for x in list:
        if x not in value_count:
            value_count[x]=0
        value_count[x]+=1
    return value_count

def predict(review):
    probability = []
    words_in_review_set = set(review)
    words_counts = count_value(review)
    for label in range(11):
        prob = 0
        for word in words_in_review_set:
            if word not in words_index_dict:
                continue
            prob+=log(tfidf[label][words_index_dict[word]]*words_counts[word]+1)
        prob *= label_count[label]/label_count[-1]
        probability.append(prob)
    return probability.index(max(probability))

In [ ]:
# Download data for deploying prediction application in Fask Using Pythonanywhere
with open('tfidf.json','w') as f_obj:
  json.dump(tfidf,f_obj)

with open('label_count.json','w') as f_obj:
  json.dump(label_count,f_obj)

with open('words_index_dict.json','w') as f_obj:
  json.dump(words_index_dict,f_obj)

In [17]:
correct = 0
for i in range(len(x_test)):
    if predict(x_test[i]) == y_test[i]:
        correct+=1
accuracy = correct/len(x_test)
print("Accuracy = ", accuracy)

Accuracy =  0.2647003070857186
